# Introduction 

You are part of a crisis response team, and you want to identify how hospitals have been responding to crash collisions in New York City.

<center>
<img src="https://i.imgur.com/wamd0n7.png" width="450"><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [ ]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

You'll use the `embed_map()` function to visualize your maps.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Visualize the collision data.

Run the code cell below to load a DataFrame `collisions_full` tracking major motor vehicle collisions in 2013-2018.

In [ ]:
collisions_full = pd.read_csv("../input/geospatial-course-data/NYPD_Motor_Vehicle_Collisions.csv")
collisions_full.dropna(subset=["LATITUDE"], inplace=True)
collisions_full = gpd.GeoDataFrame(collisions_full,
                                   geometry=gpd.points_from_xy(collisions_full["LONGITUDE"], 
                                                               collisions_full["LATITUDE"]))
collisions_full.crs = {'init' :'epsg:4326'}
collisions_full.geometry = collisions_full.geometry.to_crs(epsg=2263)

cols_to_check = ['NUMBER OF PERSONS INJURED', 'NUMBER OF PERSONS KILLED']
collisions = collisions_full.loc[collisions_full[cols_to_check].sum(axis=1)>0].copy()

Use the "LATITUDE" and "LONGITUDE" columns to create an interactive map to visualize the collision data.  What kind of map do you think is most effective?

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start=11) 

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)

embed_map(m, "m_1.html")

### 2) Understand hospital coverage.

Run the next code cell to load the hospital data.

In [ ]:
hospitals = gpd.read_file("../input/geospatial-course-data/nyu_2451_34494/nyu_2451_34494.shp")
hospitals['latitude'] = hospitals.geometry.to_crs(epsg=4326).y
hospitals['longitude'] = hospitals.geometry.to_crs(epsg=4326).x
hospitals.head()

Use the "latitude" and "longitude" columns to visualize the hospital locations. 

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start=11) 

for idx, row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']], popup=row['name']).add_to(m)
        
embed_map(m, "m1.html")

### 3) When was the closest hospital more than 10 kilometers away?

Create a DataFrame `outside_range` containing rows from `collisions` with crashes that occurred more than 10 km away from the closest hospital.

In [ ]:
coverage = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
my_union = coverage.geometry.unary_union

outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: my_union.contains(x))]

percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

# Check your answer
#q_4.check()

### 4) Make a recommender.

When collisions occur in distant locations, it becomes even more vital that injured persons are transported to the nearest available hospital.

With this in mind, you decide to create a recommender that takes the location of the crash as input and returns the name, latitude, and longitude of the closest hospital.

In [ ]:
def best_hospital(collision_location):
    idx_min = hospitals.geometry.distance(collision_location).idxmin()
    my_hospital = hospitals.iloc[idx_min]
    name = my_hospital["name"]
    latitude = my_hospital["latitude"]
    longitude = my_hospital["longitude"]
    return pd.Series({'name': name, 'lat': latitude, 'long': longitude})

# Test your function: this should suggest CALVARY HOSPITAL INC
best_hospital(outside_range.geometry.iloc[0])

### 5) Which hospital is under the highest demand?

Considering only collisions in the `outside_range` DataFrame, which hospital is most recommended?  

Your answer should be a Python string that exactly matches the name of the hospital returned by the function you created in **4)**.

In [ ]:
highest_demand = outside_range.geometry.apply(best_hospital).name.value_counts().idxmax()

# Check your answer
#q_5.check()

### 6) Where should the city construct new hospitals?

Run the next code cell (without changes) to visualize hospital locations, in addition to collisions that occurred more than 10 kilometers away from the closest hospital. 

In [ ]:
m = folium.Map(location=[40.7, -74], zoom_start=11) 

folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
folium.LatLngPopup().add_to(m)

embed_map(m, 'm1.html')

Click anywhere on the map to see a pop-up with the corresponding location in latitude and longitude.

The city of New York reaches out to you for help with deciding locations for two brand new hospitals.  They specifically want your help with identifying locations to bring the calculated percentage from step **3)** to less than ten percent.  Using the map (and without worrying about zoning laws (or what potential buildings would have to be removed in order to build the hospitals), can you identify two locations that would help the city accomplish this goal?  

#### Part A

Put the proposed latitude and longitude for hospital 1 in `lat_1` and `long_1`, respectively.  (Likewise for hospital 2.)

In [ ]:
# Your answer here: proposed location of hospital 1
lat_1 = 40.6714
long_1 = -73.8492

# Your answer here: proposed location of hospital 2
lat_2 = 40.6702
long_2 = -73.7612

# Check your answer
#q_6.a.check()

#### Part B

Run the next code cell without changes to see the effect of the new hospital.

Would the two new hospitals bring the percentage to less than ten percent?  If not, return to **Part A** of this question, and try again.

In [ ]:
new_df = pd.DataFrame(
    {'Latitude': [lat_1, lat_2],
     'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)

new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]

print(len(new_outside_range)/len(collisions))

# Did you help the city to meet its goal?
# q_6.b.check()

m = folium.Map(location=[40.7, -74], zoom_start=11) 

folium.GeoJson(coverage.geometry.to_crs(epsg=4326)).add_to(m)
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(m)
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m)
HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m)
folium.LatLngPopup().add_to(m)

embed_map(m, 'm1.html')

# Congratulations!

You have just completed the Geospatial micro-course!  Return to the Learn homepage to view your completion certificate.